In [ ]:
pip install requests

In [1]:
import time
import requests
import json
from datetime import datetime

# 定义请求的URL
url = "https://openneuro.org/crn/graphql"

# 定义请求头
headers = {
    "accept": "*/*",
    "accept-language": "zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6",
    "baggage": "sentry-environment=production,sentry-release=openneuro-app%404.29.9,sentry-public_key=b8eb90670f3386206200aa89e0d91076,sentry-trace_id=bca171fee411469d8b471a1079ffd52e,sentry-sample_rate=1,sentry-sampled=true",
    "content-type": "application/json",
    "cookie": "_ga=GA1.2.606643952.1727407142; _ga_MJNMY6D3FX=GS1.1.1731396714.2.1.1731397800.60.0.0",
    "origin": "https://openneuro.org",
    "priority": "u=1, i",
    "referer": "https://openneuro.org/search/modality/mri?query=%7B%22modality_selected%22%3A%22MRI%22%7D",
    "sec-ch-ua": '"Microsoft Edge";v="131", "Chromium";v="131", "Not_A Brand";v="24"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Windows"',
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "sentry-trace": "cfd7319aaa21431d844c3b9831e11dbd-9a5da24d2800d7b7-1",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0"
}

# 定义初始游标
cursor = "WzAsImRzMDA1NTk1Il0="

# 创建结果存储JSON文件
filename = f"result_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
result_file = open(filename, "w")

while True:

    # 定义查询字符串
    query = "query advancedSearchDatasets($query: JSON!, $cursor: String, $allDatasets: Boolean, $datasetType: String, $datasetStatus: String, $sortBy: JSON) {\n  datasets: advancedSearch(\n    query: $query\n    allDatasets: $allDatasets\n    datasetType: $datasetType\n    datasetStatus: $datasetStatus\n    sortBy: $sortBy\n    first: 25\n    after: $cursor\n  ) {\n    edges {\n      id\n      node {\n        id\n        created\n        uploader {\n          id\n          name\n          orcid\n        }\n        public\n        metadata {\n          ages\n        }\n        latestSnapshot {\n          size\n          tag\n          summary {\n            modalities\n            secondaryModalities\n            sessions\n            subjects\n            subjectMetadata {\n              participantId\n              age\n              sex\n              group\n            }\n            tasks\n            size\n            totalFiles\n            dataProcessed\n            pet {\n              BodyPart\n              ScannerManufacturer\n              ScannerManufacturersModelName\n              TracerName\n              TracerRadionuclide\n            }\n          }\n          description {\n            Name\n            Authors\n          }\n        }\n        analytics {\n          views\n          downloads\n        }\n        snapshots {\n          id\n          created\n          tag\n        }\n      }\n    }\n    pageInfo {\n      startCursor\n      endCursor\n      hasPreviousPage\n      hasNextPage\n      count\n    }\n  }\n}"

    # 定义查询变量
    variables = {
        "query": {
            "bool": {
                "filter": [{"match": {"latestSnapshot.summary.modalities": {"query": "MRI"}}}]
            }
        },
        "sortBy": None,
        "allDatasets": False,
        "datasetType": "All Public",
        "datasetStatus": None,
        "cursor": cursor
    }

    # 构建请求数据
    data = {
        "operationName": "advancedSearchDatasets",
        "variables": variables,
        "query": query
    }

    # 发送POST请求
    response = requests.post(url, headers=headers, json=data)

    # 解析响应数据
    if response.status_code == 200:
        result = response.json()
        # 处理数据，这里只是简单打印
        for edge in result["data"]["datasets"]["edges"]:
            # append json line to file
            result_file.write(json.dumps(edge) + "\n")

        # 获取下一页游标
        cursor = result["data"]["datasets"]["pageInfo"]["endCursor"]
        if not result["data"]["datasets"]["pageInfo"]["hasNextPage"]:
            break
    else:
        print(f"请求失败，状态码: {response.status_code}")
        break